In [1]:
import pandas as pd 
import numpy as np
from keras.preprocessing import utils
from keras import models

Using TensorFlow backend.


In [30]:
import json
from itertools import chain
import os

data = []

for file in os.listdir('../data/patents_parsed/'):
    with open(f'../data/patents_parsed/{file}', 'rt') as fin:
        data.append([json.loads(l) for l in fin])
        
        
data = list(chain(*data))
data = [r for r in data if r[0] is not None]
data = [r for r in data if len(r[0]) >= 200]
len(data)

6382

In [31]:
abstracts = [d[0] for d in data]
titles = [d[1] for d in data]

chars = []
for abstract in abstracts:
    for ch in abstract:
        chars.append(ch)
        
chars = set(chars)
len(chars)

147

In [32]:
import gensim

In [50]:
from gensim.test.utils import common_texts, get_tmpfile
>>> from gensim.models import Word2Vec

In [53]:
model.wv.vocab

{'human': <gensim.models.keyedvectors.Vocab at 0x7f649dd54550>,
 'interface': <gensim.models.keyedvectors.Vocab at 0x7f649dd54f28>,
 'computer': <gensim.models.keyedvectors.Vocab at 0x7f649dd54518>,
 'survey': <gensim.models.keyedvectors.Vocab at 0x7f649dd54d30>,
 'user': <gensim.models.keyedvectors.Vocab at 0x7f649dd54588>,
 'system': <gensim.models.keyedvectors.Vocab at 0x7f649dd54e48>,
 'response': <gensim.models.keyedvectors.Vocab at 0x7f649dd544a8>,
 'time': <gensim.models.keyedvectors.Vocab at 0x7f649dd54cf8>,
 'eps': <gensim.models.keyedvectors.Vocab at 0x7f649dd54c88>,
 'trees': <gensim.models.keyedvectors.Vocab at 0x7f649dd54b00>,
 'graph': <gensim.models.keyedvectors.Vocab at 0x7f649dd54da0>,
 'minors': <gensim.models.keyedvectors.Vocab at 0x7f649dd54278>}

In [60]:
len(common_texts)

9

In [61]:
abstracts_split = [a.split() for a in abstracts]

In [62]:
len(abstracts_split)

6382

In [54]:
model = Word2Vec(common_texts, size=100, window=5, min_count=1, workers=4)
model.wv.most_similar(positive = ['human'])

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('minors', 0.12259739637374878),
 ('trees', 0.11897511780261993),
 ('response', 0.08737078309059143),
 ('time', 0.07807287573814392),
 ('computer', 0.04872375726699829),
 ('eps', 0.04728245735168457),
 ('user', 0.011833075433969498),
 ('survey', 0.004004709422588348),
 ('interface', -0.0035754162818193436),
 ('graph', -0.005381472408771515)]

In [66]:
model = gensim.models.Word2Vec(abstracts_split, size = 100, window = 5,
                               workers=5)

In [68]:
len(model.wv.vocab)

9282

In [72]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words = 50000)
pad_sequences(abstracts)

ValueError: invalid literal for int() with base 10: 'A computer implemented method facilitates the capability for shop re-work orders to be effectively scheduled, knowing the time and location of item availability that is needed to correct the problem 

In [86]:
tokenizer.fit_on_texts(abstracts)

tokens = tokenizer.texts_to_sequences(abstracts)

In [87]:
len(tokens)

6382

In [88]:
len(tokens[0])

115

In [90]:
wtokens = pad_sequences(tokens, maxlen=100)

In [94]:
len(tokenizer.word_counts)

16075

In [92]:
wtokens[0]

array([ 3052,  6400,     1,    48,     4,   211,     3,   467,  2253,
          20,     8,  1581,     5,   954,     1,   334,  1372,     7,
           1,  1192,   789,  5452,  2497,     1,    13,   270,   762,
        2207,   302,    11,   409,    20,    32,    21,    58,     7,
           1,  5452,  2497,     4,   115,  4138,  3636,  4139,   154,
          20,     1,  1192,   789,  5452,  2497,    32,    21,  3052,
         124,   302,    11,   409,    17,   120,   451,     1,    13,
         115,   391,     5,     1,   899,  1240,    13,     5,  1192,
        1424,   409,    37,  1783,   899,  1240,    62,    51,    24,
           1, 11487,   899,  1894,  1240,    62,     4,   119,  2794,
        3636,  4139,   154,    20,  1192,   789,  2497,    32,    21,
        3052], dtype=int32)

In [84]:
tokens.shape

(6382, 563)

In [70]:
model.wv.get_vector('computer')

array([ 2.3627121 ,  0.28522757, -1.9136485 ,  1.8326861 , -0.7968364 ,
       -0.20501785, -0.9264961 , -1.9875044 ,  0.27362773, -0.31534263,
        1.2691286 ,  0.6617368 ,  0.49398565,  0.05749457,  0.55550814,
       -0.02495062, -2.2711115 , -0.5858653 , -1.3785905 ,  1.0760725 ,
        1.5409336 , -1.6754682 , -2.5721993 , -0.37432864,  0.09750904,
        0.45855656, -1.1702099 ,  1.507866  ,  0.8396442 ,  0.8535214 ,
       -1.1247499 , -2.0709488 ,  1.6000314 ,  1.8581088 , -1.1098697 ,
        0.1850865 , -2.1839714 , -1.438979  ,  1.799372  , -0.21686904,
       -1.9578377 , -0.7918767 ,  2.4785118 , -2.4967287 ,  0.05557762,
        0.16212144, -0.3192573 ,  0.4079158 ,  0.1613868 ,  0.08039954,
       -1.7304522 , -2.4170291 , -0.803511  , -0.25887647,  0.03718725,
       -0.9700573 ,  1.1086026 , -0.49431643, -0.4532563 , -0.21016037,
       -0.67295086, -1.5961703 , -1.1284814 , -1.5268633 , -0.18875024,
       -2.005985  ,  0.6867363 , -0.58384347,  0.7803836 , -0.49

In [47]:
model.corpus_count

6382

In [41]:
for x in model.vocabulary:
    print(x)

TypeError: 'Word2VecVocab' object is not iterable

In [40]:
model.wv.get_vector('c')

array([-0.05123514,  0.05868444,  0.44441643, -0.24288712, -0.46800336,
       -0.48196575,  0.24043837, -0.17845932,  0.20962243, -0.3418752 ,
       -0.5508718 , -0.10630816,  0.2522615 ,  0.12727764,  0.3020801 ,
       -0.16214202, -0.22684409, -0.18211724, -0.09966335,  0.11629434,
       -0.41902462, -0.3975493 , -0.24020046, -0.08236235,  0.16478544,
       -0.01692858, -0.1322483 ,  0.4525788 ,  0.17023714,  0.10563406,
       -0.17472696,  0.04847569, -0.5994605 ,  0.0663418 , -0.26894736,
        0.15451895, -0.01773221, -0.28634632, -0.15914764,  0.2864715 ,
        0.29430082, -0.10797698, -0.34496436, -0.25433847, -0.01160682,
        0.06273456, -0.05913435,  0.14484854, -0.3306361 , -0.02652841,
       -0.55654407, -0.03328067, -0.19072019, -0.0921234 ,  0.40265986,
       -0.8008943 , -0.04934891, -0.07889152,  0.30807775, -0.34875455,
        0.68752664,  0.45849988,  0.1568102 , -0.002287  ,  0.24896935,
       -0.31700772,  0.22269525,  0.37641865, -0.15583992, -0.44

In [2]:
model = models.load_model('../models/first_rnn.h5')

In [4]:
utils.plot_model(model, show_shapes=True)

In [21]:
l = model.get_layer(index = 3)
w = l.get_weights()
len(w)

3

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, None, 147)         0         
_________________________________________________________________
lstm_layer_0 (LSTM)          (None, None, 512)         1351680   
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 512)         0         
_________________________________________________________________
lstm_layer_1 (LSTM)          (None, None, 512)         2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 512)         0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 147)         75411     
Total params: 3,526,291
Trainable params: 3,526,291
Non-trainable params: 0
_________________________________________________________________


In [28]:
w[0].shape, w[1].shape, w[2].shape

((512, 2048), (512, 2048), (2048,))

In [10]:
w

[]